<a href="https://colab.research.google.com/github/PJ26487/Data_Analytics/blob/main/Product_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing the libraries
#scraping data from competitors and websites 
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd

import spacy 
nlp= spacy.load("en_core_web_sm")
from spacy import displacy
headers ='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'

In [ ]:
#extracting a list of all the links at ashley store

j = 0
link_list = []
name_list= []
price_list = []
while j<20:
  page = j*16
  url = f'https://www.ashleyfurniture.com/unxsearch/?q=pillow&start={page}&sz=16'
  
  r = requests.get(url,headers)
  print(r)
  soup= BeautifulSoup(r.content,'html.parser')
  #print(soup)

  #for finding the href tags and the links
  a = soup.find_all('a',class_='name-link')
  #print(a)
  for item in a:
    link = item.get('href')
    link_list.append(link)
    name= item.get('title')
    name_list.append(name)
  print(link_list)
  print(name_list)

  width_list =[]
  depth_list =[]
  height_list =[]
  weight_list =[]

  price_div = soup.find_all('div',class_='product-pricing ')
  for price in price_div:
    price_pro = price.find('div',class_='product-sales-price').text
    price_list.append(price_pro[2:7])
  print(price_list)
  j=j+1

for i in range(0,len(link_list)):
  url1= link_list[i]
  r= requests.get(url1,headers)
  soup = BeautifulSoup(r.content,'html.parser')
  divs = soup.find_all('div',class_='pdp-tabs js-pdp-tabs')
  #print(divs)
  for item in divs:
    koko=item.find_all('li')
    #print(koko)
    for element in koko:
      koko_text= element.text
      #print(koko_text)
      if 'Width' in koko_text:
        width_list.append(koko_text[7:12])
      elif 'Depth' in koko_text:
        depth_list.append(koko_text[7:12])
      elif 'Height' in koko_text:
        height_list.append(koko_text[7:12])
    weight= item.find('div',class_='detail-row').text
    weight_list.append(weight[8:10])

print(width_list)
print(depth_list)
print(height_list)
print(weight_list)

<Response [200]>
['https://www.ashleyfurniture.com/p/sensorpedic_sofloft_extra_firm_density_standard_pillow_2_pack/M60000452.html', 'https://www.ashleyfurniture.com/p/sensorpedic_sofloft_medium_density_standard_pillow_2_pack/M60000446.html', 'https://www.ashleyfurniture.com/p/sensorpedic_sofloft_firm_density_standard_pillow_2_pack/M60000449.html', 'https://www.ashleyfurniture.com/p/nourison_mina_victory_20_x_20_throw_pillow/A600009252.html', 'https://www.ashleyfurniture.com/p/jasmen_pillow/A1000838P.html', 'https://www.ashleyfurniture.com/p/ricker_pillow/A1000804P.html', 'https://www.ashleyfurniture.com/p/sensorpedic_luxury_extraordinaire_gusseted_gel-infused_memory_foam_oversized_bed_pillow/M60000476.html', 'https://www.ashleyfurniture.com/p/sensorpedic_coolmax_400_thread_count_cotton-rich_fiber_jumbo_bed_pillow_2_pack/M60000487.html', 'https://www.ashleyfurniture.com/p/cortnie_pillow/A1000953P.html', 'https://www.ashleyfurniture.com/p/bedgear_level_3.0_pillow/M60000262.html', 'https:

In [ ]:
#creating a dataframe from the lists
print(len(width_list))
print(len(link_list))
print(len(height_list))
print(len(weight_list))
print(len(price_list))
#creating a df for getting all the link in a list 
Pillow_df = pd.DataFrame({'Product Name':name_list[0:153],'Price  USD':price_list[0:153],'Weight lbs':weight_list[0:153],'Width inches':width_list[0:153],'Depth inches':depth_list[0:153],'Height inches':height_list[0:153]})
print(Pillow_df)

In [ ]:
#sending the df off to google sheets
#will require authentication

from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials as GC
gc = gspread.authorize(GC.get_application_default())

# create, and save df
from gspread_dataframe import set_with_dataframe
title = 'Pillow_Links_Ashley'
gc.create(title)  # if not exist
sheet = gc.open(title).sheet1
set_with_dataframe(sheet,Pillow_df) 
# include_index=False, include_column_header=True, resize=False